In [35]:
import napari
import numpy as np

import numpy as np
import napari
from matplotlib.colors import LinearSegmentedColormap
import imageio

In [4]:
data = {}
for i, cp in enumerate(['checkpoint_0',  'checkpoint_300',  'checkpoint_5660']):

    path = '/Users/henrypinkard/Desktop/figure_3_eric_data/e2e_full_volume_psfs/' + cp
    path += '_psf.npy'
    img = np.load(path)

In [48]:


cmap = LinearSegmentedColormap.from_list('prob', [[0,0,0], 'lime'])
colors = cmap(np.linspace(0, 1, 256))  # Get RGBA colors from the colormap
# Napari expects a colormap as a list of colors or an Nx4 numpy array and a control points array
napari_colormap = napari.utils.Colormap(colors)



z_anisotropy = 4  # Z pixels are 4 times larger than X and Y pixels

# Assuming 'img', 'cp', and 'napari_colormap' are defined as before
viewer = napari.view_image(img, name=cp, colormap=napari_colormap,
                           scale=[z_anisotropy, 1, 1],
                           contrast_limits=[0, .05])
viewer.dims.ndisplay = 3

# Adjust the corners calculation to take z_anisotropy into account
shape = img.shape
z, y, x = np.indices((2, 2, 2))
z = z * (shape[0] - 1) * z_anisotropy  # Apply z_anisotropy scaling to Z coordinates
y = y * (shape[1] - 1)
x = x * (shape[2] - 1)
corners = np.stack([z.flatten(), y.flatten(), x.flatten()], axis=1)

# Define the edges of the box
edges = [
    [0, 1], [1, 3], [3, 2], [2, 0],  # Bottom
    [4, 5], [5, 7], [7, 6], [6, 4],  # Top
    [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical
]

# Add edges as lines to draw the bounding box
exclude = [6, 7, 10]
for i, (start, end) in enumerate(edges):
    if i in exclude:
        continue
    viewer.add_shapes([corners[start], corners[end]], shape_type='line', edge_width=.05,
                      edge_color='white', name='Bounding Box')

# Assuming 'viewer' is your Napari viewer instance
viewer.camera.center = (20.0, 49.5, 49.5)
viewer.camera.zoom = 5.252703239519902
viewer.camera.angles = (-29.726019851146486, 55.511697835359755, 53.177248872700154)
viewer.camera.perspective = 0.0
viewer.camera.interactive = True


In [37]:
shot = viewer.screenshot(canvas_only=True)
imageio.imwrite(f'/Users/henrypinkard/Desktop/figure_psfs_tmp/{cp}_psf.png', shot)